In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


In [12]:

def evaluate_model_per_location_and_hour(model, test_loader, device, location_indices_test, output_seq_len=72):
    """
    Evaluate the model witht the needed metrics
    """
    model.eval()
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            preds = (outputs > 0.5).float()
            
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())
    
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    preds_flat, labels_flat = all_preds.flatten(), all_labels.flatten()
    
    # All metrics Required
    accuracy = (preds_flat == labels_flat).mean()
    precision = precision_score(labels_flat, preds_flat, average="binary")
    recall = recall_score(labels_flat, preds_flat, average="binary")
    f1 = f1_score(labels_flat, preds_flat, average="binary")
    conf_matrix = confusion_matrix(labels_flat, preds_flat)
    
    # Per-location metrics, added other metrics to the old function
    unique_locations = np.unique(location_indices_test)
    location_metrics = {}
    for loc in unique_locations:
        idx = (location_indices_test == loc)
        preds_loc, labels_loc = all_preds[idx].flatten(), all_labels[idx].flatten()
        location_metrics[int(loc)] = {
            "accuracy": (preds_loc == labels_loc).mean(),
            "precision": precision_score(labels_loc, preds_loc, average="binary"),
            "recall": recall_score(labels_loc, preds_loc, average="binary"),
            "f1": f1_score(labels_loc, preds_loc, average="binary"),
            "confusion_matrix": confusion_matrix(labels_loc, preds_loc)
        }
    
    # Per-hour metrics, added other metrics to the old function
    hour_metrics = []
    for hour in range(output_seq_len):
        preds_hour, labels_hour = all_preds[:, hour], all_labels[:, hour]
        hour_metrics.append({
            "hour": hour + 1,
            "accuracy": (preds_hour == labels_hour).mean(),
            "precision": precision_score(labels_hour, preds_hour, average="binary"),
            "recall": recall_score(labels_hour, preds_hour, average="binary"),
            "f1": f1_score(labels_hour, preds_hour, average="binary"),
            "confusion_matrix": confusion_matrix(labels_hour, preds_hour)
        })
    
    return {
        "overall": {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "confusion_matrix": conf_matrix
        },
        "per_location": location_metrics,
        "per_hour": hour_metrics
    }


In [13]:
def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=[0,1], yticklabels=[0,1])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(title)
    plt.show()
